In [1]:
%load_ext autoreload
%env CUDA_VISIBLE_DEVICES = 2 python benchmark.ipynb

env: CUDA_VISIBLE_DEVICES=2 python benchmark.ipynb


In [2]:
import pandas as pd
import torch.nn as nn

from benchmark import tracker

%autoreload 2
pd.set_option("display.precision", 2)

In [3]:
device = 'cuda'
device

'cuda'

In [4]:
shape = (1, 64, 128, 128)

module_collection = {'conv_64_512_1x1': nn.Conv2d(kernel_size=1, in_channels=64, out_channels=512), 
                     'conv_64_512_3x3': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512), 
                     'bottleneck_64_512_3x3': nn.Sequential(*[nn.Conv2d(kernel_size=1, in_channels=64, out_channels=32), 
                                                             nn.Conv2d(kernel_size=3, in_channels=32, out_channels=512),
                                                              ]),
                      'conv_64_512_3x3_g2': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=2),
                      'conv_64_512_3x3_g8': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=8),
                      'conv_padding': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512, padding=1),
                      'conv_nn_Padding': nn.Sequential(nn.ZeroPad2d(1),
                                                       nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512))
                    }

In [5]:
# contiguous
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters', 'time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, memory_unit='KB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.38,0.04,0.33,0.02,33786.37,131.07,545259520.0,33280.0,18445.25
conv_64_512_3x3,0.93,1.04,1.6,1.25,36835.33,1179.65,4690151424.0,295424.0,5339.87
bottleneck_64_512_3x3,1.11,4.97,1.85,2.31,37294.08,2686.98,2383218688.0,150048.0,5011.83
conv_64_512_3x3_g2,2.01,0.19,2.52,0.03,34849.28,3211.26,11956733952.0,803328.0,5938.68
conv_64_512_3x3_g8,0.69,0.07,1.53,0.77,32983.04,802.82,2994898944.0,201216.0,4198.5
conv_padding,0.8,0.21,1.15,1.89,36835.33,91357.18,4840226816.0,295424.0,4691.98
conv_nn_Padding,1.12,3.51,1.36,0.89,41291.78,1179.65,4840226816.0,295424.0,5198.04


In [6]:
# channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, channels_last=True, memory_unit='MB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.43,0.05,1.02,3.76,38.24,35.65,545259520.0,33280.0,4333.77
conv_64_512_3x3,1.07,0.25,1.42,1.32,39.03,35.65,4690151424.0,295424.0,5717.19
bottleneck_64_512_3x3,0.78,0.05,2.22,0.17,41.57,35.65,2383218688.0,150048.0,5418.13
conv_64_512_3x3_g2,1.92,0.12,13.45,0.54,39.24,41.11,11956733952.0,803328.0,18013.13
conv_64_512_3x3_g8,1.07,0.1,12.0,0.22,37.37,36.45,2994898944.0,201216.0,10108.63
conv_padding,1.0,0.03,1.86,0.04,40.04,35.65,4840226816.0,295424.0,1737.72
conv_nn_Padding,1.07,0.08,1.21,0.08,44.3,35.65,4840226816.0,295424.0,5190.03


In [ ]:
# amp
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, amp=True, memory_unit='GB')

module_collection_stats

In [ ]:
# amp + channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, channels_last=True, amp=True, memory_unit='B')

module_collection_stats